In [1]:
import os, ast
import pandas as pd
import numpy as np
import scipy
import matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt
import itertools as it
import colorbrewer
import json
import collections
from hs_vervet.tools import hs_pandas as hpd

eu = os.path.expanduser
jn = os.path.join
meta_dir = eu("~/vervet_project/metadata")
var_ana_dir = eu("~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data")
manuscript_dir = eu("~/VervetPopgen/Manuscript/VervetPopgen_man")
manuscript_dir2 = eu("~/VervetPopgen/Manuscript/VervetPopgen_man_0_2")
man_ana_dir = eu("~/vervet_project/analyses/20150520_popgen_analysis_UG/_data")
xpclr_ana_dir = eu("~/vervet_project/analyses/20150528_UG_xpclr/_data") 
assoc_dir = eu("~/vervet_project/data/gene_associations")
figure_dir = eu("~/VervetPopgen/Figures")
google_xpclr_dir = eu("~/hannes.svardal@gmail.com/Nordborg lab/Vervet monkeys/GMI_vervetpopgen_log/data/xpclr")
vervet_drive_dir = eu("~/Akademisches/Projects/VervetPopgen")
google_man_dir = "/home/GMI/hannes.svardal/hannes.svardal@gmail.com/Nordborg lab/Vervet monkeys/GMI_vervetpopgen_log/manuscript/163_pop_manuscript/"
ballet_dir = eu("~/vervet_project/analyses/20150601_UG_ballet/_data")
pops = ["aet","cyn","pyg","sab","tan"]
colors = json.load(open(jn(meta_dir,'colors.json')))

In [10]:
from enrichme import enrichme as em
from enrichme import pandas_util as pu

## Test different methods with random scores that are in LD

## Denisovan admixture

In [5]:
denisovan_s = pd.read_csv(eu("~/Akademisches/Projects/Enrichme/Test_data/Denisowan_admixture/Denisovan_scores_hannes.bed"), sep="\t",index_col=[0,1],
               usecols=[0,1,3],squeeze=True,header=False,names=["chrom","pos","score"])

In [51]:
human_gene_df = pd.read_csv(eu("~/vervet_project/data/other_species/human/human_gene_positions_hg38.5.tsv"),
                            sep='\t',names=['chrom','start','end','gene_symbol','ensembl_id'],skiprows=1)
human_gene_df_chrom = human_gene_df[human_gene_df['chrom'].apply(lambda s: s in [str(i) for i in range(1,24)] + ['X'])]
human_gene_df_chrom['chrom'] = human_gene_df_chrom['chrom'].apply(lambda s: 'chr'+s)
human_gene_df_chrom.set_index(['chrom','start'],inplace=True)

In [87]:
human_gene_df_chrom.reset_index().set_index('gene_symbol').ix[['OR1D2','OR2B6']]

,chrom,start,end,ensembl_id
gene_symbol,,,,
OR1D2,chr17,3092058,3092996,ENSG00000184166
OR2B6,chr6,27957241,27958182,ENSG00000124657


In [68]:
with open(eu("~/VervetPopgen/scratch/all_human_genes_enembl.txt"),'w') as f:
    for g in human_gene_df_chrom["ensembl_id"].values:
        f.write(g+'\n')

### define a cutoff

In [97]:
cutoff = 5
top_s = denisovan_s[denisovan_s>cutoff]
candidate_list = pu.get_features(top_s,human_gene_df_chrom,max_dist=5000,feature_name="ensembl_id")
with open(eu("~/VervetPopgen/scratch/denisova_gt_{}.txt".format(cutoff)),'w') as f:
    for g in candidate_list:
        f.write(g+'\n')

In [93]:
np.random.rand()

0.13769536364755341

In [106]:
cutoff = 5
shift = np.random.rand()
print shift
shift_s = em.shift_rod(denisovan_s, np.random.rand())
top_s = shift_s[shift_s>cutoff]
candidate_list = pu.get_features(top_s,human_gene_df_chrom,max_dist=5000,feature_name="ensembl_id")
with open(eu("~/VervetPopgen/scratch/denisova_shift_gt_{}.txt".format(cutoff)),'w') as f:
    for g in candidate_list:
        f.write(g+'\n')

0.345538163364


In [103]:
denisova_score_per_gene = pu.data_per_feature(denisovan_s,human_gene_df_chrom, feature_name='ensembl_id')

In [79]:
max_per_gene = denisova_score_per_gene.groupby("ensembl_id").max()
mean_per_gene = denisova_score_per_gene.groupby("ensembl_id").mean()

In [84]:
with open(eu("~/VervetPopgen/scratch/human_ranked_by_denisowa_score_max.txt"),'w') as f:
    f.write("\n".join(max_per_gene.sort("score",ascending=False,inplace=False).index)+'\n')

In [85]:
with open(eu("~/VervetPopgen/scratch/human_ranked_by_denisowa_score_mean.txt"),'w') as f:
    f.write("\n".join(mean_per_gene.sort("score",ascending=False,inplace=False).index)+'\n')

## UK 10k GWAS

In [110]:
gwas_s = pd.read_csv(eu("~/Akademisches/Projects/Enrichme/Test_data/human_gwas/HDL.bedGraph"),
                     sep='\t',usecols=[0,1,3],names=['chrom','pos','score'],squeeze=True, index_col=[0,1])

In [114]:
cutoff = 5
top_s = gwas_s[gwas_s>cutoff]
candidate_list = pu.get_features(top_s,human_gene_df_chrom,max_dist=5000,feature_name="ensembl_id")
with open(eu("~/VervetPopgen/scratch/hdl_gt_{}.txt".format(cutoff)),'w') as f:
    for g in candidate_list:
        f.write(g+'\n')

In [115]:
len(candidate_list)

65